# rhino3dm exercise 02

## read project 3dm file

In [1]:
import rhino3dm

In [2]:
rhino_file = "./data/tw_nongke.3dm"
model = rhino3dm.File3dm.Read(rhino_file)
print('3dm object: ', model)

3dm object:  <rhino3dm._rhino3dm.File3dm object at 0x0000023058927070>

## read `Layer`

In [3]:
layer_table = model.Layers
print('layer table: ', layer_table)
print('num of layers: ', len(layer_table), '\n')

for idx, layer in enumerate(iter(layer_table)):
    print('layer ', idx, ' Object  : ', layer)
    print('      ', idx, ' Id      : ', layer.Id)
    print('      ', idx, ' Index   : ', layer.Index)
    print('      ', idx, ' Name    : ', layer.Name)
    print('      ', idx, ' Fullpath: ', layer.FullPath)
    print('      ', idx, ' Color   : ', layer.Color)

layer table:  <rhino3dm._rhino3dm.File3dmLayerTable object at 0x000002305E83E2F0>
num of layers:  4 

layer  0  Object  :  <rhino3dm._rhino3dm.Layer object at 0x000002305E83E770>
       0  Id      :  212960d6-ad46-439d-8516-772ce85fde03
       0  Index   :  0
       0  Name    :  0
       0  Fullpath:  0
       0  Color   :  (0, 0, 0, 255)
layer  1  Object  :  <rhino3dm._rhino3dm.Layer object at 0x0000023058976CB0>
       1  Id      :  25488de8-55a5-470b-a098-ea3c8c1220b3
       1  Index   :  1
       1  Name    :  cable
       1  Fullpath:  cable
       1  Color   :  (255, 127, 0, 255)
layer  2  Object  :  <rhino3dm._rhino3dm.Layer object at 0x000002302EF9F7B0>
       2  Id      :  0428f404-44e4-47e7-ae60-72284d0a89f7
       2  Index   :  2
       2  Name    :  pv
       2  Fullpath:  pv
       2  Color   :  (0, 0, 0, 255)
layer  3  Object  :  <rhino3dm._rhino3dm.Layer object at 0x0000023058DF6070>
       3  Id      :  6f40c8fc-3378-4efb-a590-0af0bf5a85dc
       3  Index   :  3
      

## read `Objects` on the `pv` layer

In [4]:
object_table = model.Objects
print('object table: ', object_table, '\n')

for idx, obj in enumerate(iter(object_table)):
    if idx < 77 and obj.Attributes.LayerIndex == 2:
        print('object ', idx, ' Object                : ', obj)
        print('       ', idx, ' Geometry              : ', obj.Geometry)
        print('       ', idx, ' Attributes            : ', obj.Attributes)
        print('       ', idx, ' Attributes.Id         : ', obj.Attributes.Id)
        print('       ', idx, ' Attributes.Name       : ', obj.Attributes.Name)
        print('       ', idx, ' Attributes.LayerIndex : ', obj.Attributes.LayerIndex)
        print('       ', idx, ' Attributes.ObjectColor: ', obj.Attributes.ObjectColor)
        print('       ', idx, ' Attributes.Visible    : ', obj.Attributes.Visible)
        print()
    else:
        pass

object table:  <rhino3dm._rhino3dm.File3dmObjectTable object at 0x0000023058DF5730> 

object  74  Object                :  <rhino3dm._rhino3dm.File3dmObject object at 0x000002305E876EB0>
        74  Geometry              :  <rhino3dm._rhino3dm.PolylineCurve object at 0x0000023058DF71B0>
        74  Attributes            :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x0000023058DF71B0>
        74  Attributes.Id         :  36f08c55-5986-4aa2-b0b8-ae56b26b043c
        74  Attributes.Name       :  
        74  Attributes.LayerIndex :  2
        74  Attributes.ObjectColor:  (0, 0, 0, 255)
        74  Attributes.Visible    :  True

object  75  Object                :  <rhino3dm._rhino3dm.File3dmObject object at 0x000002305E8778F0>
        75  Geometry              :  <rhino3dm._rhino3dm.PolylineCurve object at 0x000002302F0446B0>
        75  Attributes            :  <rhino3dm._rhino3dm.ObjectAttributes object at 0x000002302F0446B0>
        75  Attributes.Id         :  2a5e0e48-e0ca-4d27-